In [1]:
!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform --quiet


import and initialise variables

In [2]:
from google.cloud import bigquery
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel

PROJECT_ID = "qwiklabs-gcp-02-598c62ac8986"
DATASET = "challenge_2"
TABLE = "challenge_2_embedded"
EMBEDDING_MODEL = "Embeddings"

bq_client = bigquery.Client(project=PROJECT_ID)
aiplatform.init(project=PROJECT_ID, location="us-central1")

Vector search functionality - load the results from the big query table

In [3]:
def query_bigquery_vector_search(user_question: str, top_k: int = 3):

    query = f"""
SELECT
  query.query,
  result.base.content,
  result.base.answer,
  result.distance
FROM VECTOR_SEARCH(
  TABLE `{PROJECT_ID}.{DATASET}.{TABLE}`,
  'ml_generate_embedding_result',
  (
    SELECT
      ml_generate_embedding_result AS embedding,
      '{user_question}' AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `{PROJECT_ID}.{DATASET}.{EMBEDDING_MODEL}`,
      (SELECT '{user_question}' AS content)
    )
  ),
  top_k => {top_k},
  options => '{{"fraction_lists_to_search": 1.0}}'
) AS result
"""
    results = bq_client.query(query).to_dataframe()
    return results


Retrieve the vector search results, set the context and prompt for the model and invoke the model to respond to the user's question.

In [4]:
def chatbot():
    print("Chatbot: Hello! Ask me anything about your proprietary data.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break

        results = query_bigquery_vector_search(user_input)

        if results.empty:
            print("\nChatbot: I couldn't find relevant information in my knowledge base.")
            continue

        context = ""
        for idx, row in results.iterrows():
            context += f"Q: {row['content']}\nA: {row['answer']}\n\n"

        prompt = (
            f"Based on the following proprietary Q&A data, answer the user's question:\n\n"
            f"{context}\n"
            f"User's question: {user_input}\n"
            f"Answer:"
        )

        model = GenerativeModel('gemini-2.0-flash-001')
        response = model.generate_content(prompt)
        print(f"\nChatbot: {response.text.strip()}")

chatbot()

Chatbot: Hello! Ask me anything about your proprietary data.

You: when was aurora bay founded

Chatbot: Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.

You: exit
Chatbot: Goodbye!
